# Assignment 1 - Data Science in Finance and Insurance
Submission by: Dennis Goldenberg


In [3]:
#import statements
import scipy as sc;
import numpy as np;
import matplotlib as plt;


## Generating 2000 Samples from $\chi^2(11)$ random variable

In [52]:
#Draw from chi-squared dist
np.random.seed(462001)
samples = sc.stats.chi2.rvs(df = 11, size = 2000000)
samples = samples.reshape(2000,1000)

2000

## Generating Confidence intervals for each samples
Define $t_{p, df}$ as the 100pth percentile of the $t$ distribution with $df$ degrees of freedom. Since the variance is presumed unknown, use the following formula for the 95% confidence interval for the mean of each sample:
$$\left(\bar{x} - t_{.975, 999}*\widehat{se}(x), \bar{x} + t_{.975, 999}*\widehat{se}(x)\right)$$
Note that I am using $.975$ as the percentile for the $t$-distribution since the t is symmetric and I want to omit the most extreme 2.5% of data on both tails.

In [60]:
#95% confidence interval

#Use the t distribution, since variance assumed unknown
conf_invs_lb = np.mean(samples, axis = 1) - sc.stats.t.ppf(.975, df = 999)*(np.std(samples, axis = 1, ddof = 1)/np.sqrt(len(samples[0]) - 1))
conf_invs_ub = np.mean(samples, axis = 1) + sc.stats.t.ppf(.975, df = 999)*(np.std(samples, axis = 1, ddof = 1)/np.sqrt(len(samples[0]) - 1))


0.055